In [3]:
import pandas as pd 
import pyfaidx
import pysam
import os 
from tqdm import tqdm
from helper_functions import *

refdir = "/Volumes/HNSD01/storage/ref"
outdir = "/Volumes/HNSD01/outdir"

# cancer_type = "Colorectal"
cancer_type = "Lung"
# cancer_type = "Gastric"

gene_list_version = "v0.1"
output_version = "20250918"

k = 24
additional_flanking_size = 100

path_to_main_output = os.path.join(outdir, "k-mer-feature", "data_analysis", output_version)
path_to_01_output = os.path.join(path_to_main_output, 
                                 "01_ref_kmer_from_promoter_regions", 
                                 cancer_type, 
                                 gene_list_version, 
                                 f"{k}-mer")
os.system(f"mkdir -p {path_to_01_output}")

path_to_hg19 = os.path.join(refdir, "hg19.fa")

# always get list of all up and down regulated genes.
path_to_bed_file = os.path.join(f"./custom_tss_gene_beds/{cancer_type}/{gene_list_version}/biomart/all_genes/promoter_regions_up_1000_down_1000.bed")
beddf = pd.read_csv(path_to_bed_file, sep="\t", header=None)
beddf.columns = ["chrom", "start", "end", "name"]

for i in tqdm(range(beddf.shape[0])):
    chrom = beddf.iloc[i]["chrom"]
    start = beddf.iloc[i]["start"]
    end = beddf.iloc[i]["end"]
    name = beddf.iloc[i]["name"]
    seq = get_refseq(path_to_hg19, chrom, start, end)
    ref_kmer = pd.DataFrame.from_dict(kmer_occurence_count(seq, k), orient="index").reset_index()
    ref_kmer.columns = ["kmer", "count"]
    ref_kmer = ref_kmer.sort_values(by="count", ascending=False).reset_index(drop=True)
    ref_kmer.to_csv(os.path.join(path_to_01_output, f"{chrom}_{start}_{end}_{name}_{k}-mer.csv"), index=False)

  0%|          | 0/86 [00:00<?, ?it/s]

100%|██████████| 86/86 [00:00<00:00, 358.74it/s]
